In [1]:
import numpy as np
import nltk
import json
# import Levenshtein

import moxing as mox
mox.file.shift('os', 'mox')

INFO:root:Using MoXing-v1.14.1-ddfd6c9a
INFO:root:Using OBS-Python-SDK-3.1.2


In [2]:
class AnalyseCorpus(object):

    def __init__(self, corpus):
        self.corpus = corpus  # corpus: {1:[], 2:[]}
        self.analyse = {}
        self.result = {}

    def analyse_task(self):
        for taski, ci in self.corpus.items():  # 每一个task的sentences：ci
            self.analyse[taski] = {"unigram":set(), "bigram":set(), "trigram":set(), "sentence_len":[], "sentence_num":0, "bleu_score":[]}
            
            s_num = len(ci)   # 此task下共有多少句sentence
            self.analyse[taski]["sentence_num"] = s_num
            
            for si in range(s_num):
                words = ci[si].split()
                self.analyse[taski]["sentence_len"].append(len(words))

                self.analyse[taski]["unigram"] = self.analyse[taski]["unigram"].union(set(nltk.ngrams(words,1)))
                self.analyse[taski]["bigram"] = self.analyse[taski]["bigram"].union(set(nltk.ngrams(words,2)))
                self.analyse[taski]["trigram"] = self.analyse[taski]["trigram"].union(set(nltk.ngrams(words,3)))

                if si < s_num - 1:
                    for sj in range(si + 1, s_num):
                        words2 = ci[sj].split()
                        self.analyse[taski]["bleu_score"].append(nltk.translate.bleu_score.sentence_bleu([words], words2, weights = (0.5, 0.5)))
                        
            self.result[taski] = {
                "unigram": len(self.analyse[taski]["unigram"]),
                "bigram": len(self.analyse[taski]["bigram"]),
                "trigram": len(self.analyse[taski]["trigram"]),
                "sentence_len": self.analyse[taski]["sentence_len"],
                "sentence_num": self.analyse[taski]["sentence_num"],
                "bleu_score": sum(self.analyse[taski]["bleu_score"])/len(self.analyse[taski]["bleu_score"])
            }

In [15]:
tasks_sentences = {i:[] for i in range(1,11)}
with open("s3://corpus-2/dataset/corpus_5.txt", "r", encoding='utf-8') as fp:
    for line in fp.readlines():
        lsp = line.split()
        info = lsp[0]
        sentence = " ".join(lsp[1:])
        task = int(info.split(":")[0])
        if (int(info.split(":")[2])):
            tasks_sentences[task].append(sentence)

In [16]:
analyse_corpus = AnalyseCorpus(tasks_sentences)
analyse_corpus.analyse_task()

/home/ma-user/anaconda3/envs/TensorFlow-1.8/lib/python3.6/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


In [9]:
print(analyse_corpus.result[1]["unigram"],analyse_corpus.result[1]["bigram"],analyse_corpus.result[1]["trigram"])
print(analyse_corpus.result[2]["unigram"],analyse_corpus.result[2]["bigram"],analyse_corpus.result[2]["trigram"])
print(analyse_corpus.result[1]["bleu_score"])

350 651 705
353 761 832
0.2705302334126114


In [ ]:
with open('s3://corpus-2/result/corpus_920.json','w',encoding='utf-8') as fObj:
    json.dump(analyse_corpus.result,fObj,ensure_ascii=False)

In [3]:
tasks_sentences2 = {i:[] for i in range(1,11)}
with open("s3://corpus-2/dataset/corpus_5.txt", "r", encoding='utf-8') as fp:
    for line in fp.readlines():
        lsp = line.split()
        info = lsp[0]
        sentence = " ".join(lsp[1:])
        task = int(info.split(":")[0])
        tasks_sentences2[task].append(sentence)

In [4]:
analyse_corpus2 = AnalyseCorpus(tasks_sentences2)
analyse_corpus2.analyse_task()

/home/ma-user/anaconda3/envs/TensorFlow-1.8/lib/python3.6/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


In [12]:
analyse_corpus.result

{1: {'bigram': 3757,
  'bleu_score': 0.27522585385723963,
  'sentence_len': [11,
   11,
   12,
   12,
   16,
   12,
   13,
   12,
   16,
   17,
   10,
   12,
   11,
   13,
   14,
   10,
   6,
   10,
   8,
   10,
   10,
   9,
   8,
   8,
   14,
   9,
   7,
   9,
   8,
   12,
   9,
   11,
   12,
   13,
   10,
   10,
   14,
   10,
   10,
   10,
   7,
   13,
   10,
   10,
   10,
   10,
   11,
   10,
   8,
   8,
   11,
   12,
   11,
   19,
   9,
   12,
   11,
   11,
   8,
   8,
   12,
   10,
   14,
   12,
   9,
   8,
   12,
   10,
   11,
   14,
   10,
   12,
   10,
   11,
   11,
   11,
   10,
   10,
   10,
   10,
   11,
   8,
   13,
   10,
   13,
   10,
   11,
   11,
   10,
   12,
   8,
   12,
   16,
   12,
   12,
   12,
   11,
   9,
   12,
   12,
   14,
   12,
   13,
   13,
   12,
   11,
   10,
   10,
   10,
   11,
   7,
   7,
   9,
   7,
   9,
   7,
   8,
   10,
   10,
   9,
   8,
   12,
   10,
   9,
   9,
   14,
   13,
   13,
   13,
   9,
   14,
   10,
   14,
   16,
   10,
   9,
   8,
  

In [6]:
with open('s3://corpus-2/result/corpus_14035.json','w',encoding='utf-8') as fObj:
    json.dump(analyse_corpus2.result,fObj,ensure_ascii=False)

In [9]:
analyse_corpus2.result[5]["sentence_num"]

1206